# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("Output/CityWeatherData.csv")
data_map = pd.DataFrame(file)
data_map.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Verkhnevilyuysk,97,RU,1584243389,90,63.45,120.32,18.82,3.40
1,Kavieng,99,PG,1584243389,64,-2.57,150.80,85.37,5.23
2,Kamen'-na-Obi,100,RU,1584243390,95,53.79,81.34,34.34,10.45
3,Ushuaia,40,AR,1584243390,58,-54.80,-68.30,53.60,6.93
4,Avarua,8,CK,1584243390,74,-21.21,-159.78,84.20,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


In [4]:
# Generate map
fig1 = gmaps.figure(map_type="HYBRID")

# Add locations and weights
locations = data_map[["Lat", "Lng"]]
weights = data_map["Humidity"]

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    locations=locations, weights=weights,
    max_intensity=max(data_map["Humidity"]), point_radius=5.0
)

# Add locations and weights to map
fig1.add_layer(heatmap_layer)
fig1


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create dataframe to hold only locations with ideal weather conditions

# Filter out temps only between 70-80 degrees F (not inclusive)
ideal_weather_cities = data_map.loc[(data_map["Max Temp"] < 80) & (data_map["Max Temp"] > 70)]

# Remove cities with wind speed 10 mph or greater
ideal_weather_cities = ideal_weather_cities.loc[(ideal_weather_cities["Wind Speed"] < 10)]

# Remove cities with any cloudiness
ideal_weather_cities = ideal_weather_cities.loc[(ideal_weather_cities["Cloudiness"] == 0)]


ideal_weather_cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
86,Barra de Tecoanapa,0,MX,1584243417,78,16.52,-98.75,75.36,1.21
125,Chunhuhub,0,MX,1584243438,87,19.55,-88.68,71.65,3.47
138,Chisec,0,GT,1584243455,73,15.82,-90.28,74.35,6.33
232,Nuwara Eliya,0,LK,1584243500,67,6.97,80.78,71.35,3.53
263,Kontagora,0,NG,1584243505,15,10.40,5.47,72.79,7.74
425,Bétérou,0,BJ,1584243589,66,9.20,2.26,79.18,2.44
459,Ngunguru,0,NZ,1584243610,70,-35.62,174.50,72.00,8.01
495,Mamaku,0,NZ,1584243631,55,-38.10,176.08,75.00,4.79
501,Danshui,0,CN,1584243631,43,22.80,114.47,75.99,4.47
504,Sawākin,0,SD,1584243632,59,19.11,37.33,72.64,9.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_cities

hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index()
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,86,Barra de Tecoanapa,0,MX,1584243417,78,16.52,-98.75,75.36,1.21,
1,125,Chunhuhub,0,MX,1584243438,87,19.55,-88.68,71.65,3.47,
2,138,Chisec,0,GT,1584243455,73,15.82,-90.28,74.35,6.33,
3,232,Nuwara Eliya,0,LK,1584243500,67,6.97,80.78,71.35,3.53,
4,263,Kontagora,0,NG,1584243505,15,10.40,5.47,72.79,7.74,
5,425,Bétérou,0,BJ,1584243589,66,9.20,2.26,79.18,2.44,
6,459,Ngunguru,0,NZ,1584243610,70,-35.62,174.50,72.00,8.01,
7,495,Mamaku,0,NZ,1584243631,55,-38.10,176.08,75.00,4.79,
8,501,Danshui,0,CN,1584243631,43,22.80,114.47,75.99,4.47,
9,504,Sawākin,0,SD,1584243632,59,19.11,37.33,72.64,9.44,


In [7]:
# Convert lat and lng columns to string
hotel_df["Lat"] = hotel_df["Lat"].astype(str)
hotel_df["Lng"] = hotel_df["Lng"].astype(str)

In [8]:
# Define parameters to search for hotel
target_search = "hotel"
radius = 5000

# Convert parameters to dictionary

params = {
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [9]:
# Create list to hold markers
marker_list = []

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Set city name from city's row
    name = row["City"]
    
    # Get coordinates from city's data
    city_lat = row["Lat"]
    city_lng = row["Lng"]

    # Set coordinates for parameter to city's lat and lng
    coordinates = f"{city_lat}, {city_lng}"
    
    # Add city lat and lng to parameters
    params["location"] = coordinates
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # Set a variable to hold JSON results of API call
    hotel_data = response.json()
    
    try:
        # Set variables to hold info on Hotel's Name, Lat, and Long
        hotel_name = hotel_data["results"][0]["name"]
        hotel_lat = hotel_data["results"][0]["geometry"]["location"]["lat"]
        hotel_lng = hotel_data["results"][0]["geometry"]["location"]["lng"]
        
        # If hotel not found within required radius, raise exception
    except IndexError:
        print(f"Hotel not found in the vicinity of {name}.")
        
        # Set data equal to empty
        hotel_name = "NaN"
        hotel_lat = "NaN"
        notel_lng = "NaN"

    # Create dictionary to hold hotel data
    single_hotel_data = {
        "hotel name": hotel_name,
        "hotel lat": hotel_lat,
        "hotel lng": hotel_lng
    }

    # Append dictionary to list of markers
    marker_list.append(single_hotel_data)

Hotel not found in the vicinity of Chunhuhub.
Hotel not found in the vicinity of Bétérou.
Hotel not found in the vicinity of Mamaku.


[{'hotel name': 'Hotel Garapacho',
  'hotel lat': 16.5556084,
  'hotel lng': -98.7883281},
 {'hotel name': 'NaN', 'hotel lat': 'NaN', 'hotel lng': -98.7883281},
 {'hotel name': 'Hotel Estancia De La Virgen',
  'hotel lat': 15.8172852,
  'hotel lng': -90.291585},
 {'hotel name': 'The Grand Hotel',
  'hotel lat': 6.968494499999999,
  'hotel lng': 80.7646944},
 {'hotel name': 'City Gate Inn',
  'hotel lat': 10.4017596,
  'hotel lng': 5.4726971},
 {'hotel name': 'NaN', 'hotel lat': 'NaN', 'hotel lng': 5.4726971},
 {'hotel name': 'Quality Hotel Oceans Tutukaka',
  'hotel lat': -35.6099474,
  'hotel lng': 174.5245359},
 {'hotel name': 'NaN', 'hotel lat': 'NaN', 'hotel lng': 174.5245359},
 {'hotel name': 'Grand Skylight International Hotel Huizhou',
  'hotel lat': 22.7952623,
  'hotel lng': 114.4485712},
 {'hotel name': 'اولاد خضر', 'hotel lat': 19.111225, 'hotel lng': 37.309389}]

In [12]:
# use iterrows to iterate through pandas dataframe
for x in range (0, 10):
    
    # Insert Hotel Name into Hotel Name Column
    hotel_df.iloc[x, 10] = marker_list[x]["hotel name"]

In [13]:
# Clean dataframe and markers dictionary

# Clean dataframe
# Drop NaN from dataframe and reset index
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "NaN"), :]
hotel_df = hotel_df.reset_index()
hotel_df



,level_0,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,86,Barra de Tecoanapa,0,MX,1584243417,78,16.52,-98.75,75.36,1.21,Hotel Garapacho
1,2,138,Chisec,0,GT,1584243455,73,15.82,-90.28,74.35,6.33,Hotel Estancia De La Virgen
2,3,232,Nuwara Eliya,0,LK,1584243500,67,6.97,80.78,71.35,3.53,The Grand Hotel
3,4,263,Kontagora,0,NG,1584243505,15,10.4,5.47,72.79,7.74,City Gate Inn
4,6,459,Ngunguru,0,NZ,1584243610,70,-35.62,174.5,72.00,8.01,Quality Hotel Oceans Tutukaka
5,8,501,Danshui,0,CN,1584243631,43,22.8,114.47,75.99,4.47,Grand Skylight International Hotel Huizhou
6,9,504,Sawākin,0,SD,1584243632,59,19.11,37.33,72.64,9.44,اولاد خضر


In [14]:
# Clean markers
marker_list

for x in range(len(mmarker_list)):
    if marker_list[x]["hotel name"] == "NaN":
        del marker_list[x]

[{'hotel name': 'Hotel Garapacho',
  'hotel lat': 16.5556084,
  'hotel lng': -98.7883281},
 {'hotel name': 'NaN', 'hotel lat': 'NaN', 'hotel lng': -98.7883281},
 {'hotel name': 'Hotel Estancia De La Virgen',
  'hotel lat': 15.8172852,
  'hotel lng': -90.291585},
 {'hotel name': 'The Grand Hotel',
  'hotel lat': 6.968494499999999,
  'hotel lng': 80.7646944},
 {'hotel name': 'City Gate Inn',
  'hotel lat': 10.4017596,
  'hotel lng': 5.4726971},
 {'hotel name': 'NaN', 'hotel lat': 'NaN', 'hotel lng': 5.4726971},
 {'hotel name': 'Quality Hotel Oceans Tutukaka',
  'hotel lat': -35.6099474,
  'hotel lng': 174.5245359},
 {'hotel name': 'NaN', 'hotel lat': 'NaN', 'hotel lng': 174.5245359},
 {'hotel name': 'Grand Skylight International Hotel Huizhou',
  'hotel lat': 22.7952623,
  'hotel lng': 114.4485712},
 {'hotel name': 'اولاد خضر', 'hotel lat': 19.111225, 'hotel lng': 37.309389}]

In [ ]:
# Generate map
fig2 = gmaps.figure(map_type="HYBRID")

# Add locations and weights
locations = hotel_df[["Lat", "Lng"]]
weights = hotel_df["Humidity"]

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    locations=locations, weights=weights,
    max_intensity=max(hotel_df["Humidity"]), point_radius=5.0
)

# Add locations and weights to map
fig2.add_layer(heatmap_layer)
fig2

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_cities.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map